In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
batch_sizes = ['2000', '4000', '6000']
LOGS_PATH = 'logs/experiment_1/'

training_data = []
validation_data = []
for batch_size in batch_sizes:
    
    # Load results for the different optimizers
    sgd = pd.read_csv(f'{LOGS_PATH}/SGD_{batch_size}/V0/metrics.csv')
    adam = pd.read_csv(f'{LOGS_PATH}/Adam_{batch_size}/V0/metrics.csv')
    kfac = pd.read_csv(f'{LOGS_PATH}/KFAC_dia_{batch_size}/V0/metrics.csv')
    ekfac = pd.read_csv(f'{LOGS_PATH}/KFAC_eigen_{batch_size}/V0/metrics.csv')
    
    def _extract_values(optimizer, value_key, df):
        if value_key == 'train_loss':
            return [{
                'Optimizer': optimizer,
                'bs': batch_size,
                'Negative log-likelihood': getattr(value, value_key),
                'Epoch': value.epoch
            } for value in df[[value_key, 'epoch']].itertuples(index=False)]
        else:
            return [{
                'Optimizer': optimizer,
                'bs': batch_size,
                'Negative log-likelihood': getattr(value, value_key),
                'Epoch': value.epoch
            } for value in sgd[[value_key, 'epoch']].itertuples(index=False)]
    
    # Get training data
    training_data += _extract_values('SGD with momentum', 'train_loss', sgd)
    training_data += _extract_values('Adam', 'train_loss', adam)
    training_data += _extract_values('KFAC', 'train_loss', kfac)
    training_data += _extract_values('EKFAC', 'train_loss', ekfac)
    
    # Get validation data
    validation_data += _extract_values('SGD with momentum', 'val_loss', sgd)
    validation_data += _extract_values('Adam', 'val_loss', adam)
    validation_data += _extract_values('KFAC', 'val_loss', kfac)
    validation_data += _extract_values('EKFAC', 'val_loss', ekfac)
    
# Remove NaN values (because the logger keeps entries for both validation and loss we have to do this)
training_df = pd.DataFrame(training_data).dropna()
validation_df = pd.DataFrame(validation_data).dropna()

In [ ]:
plt.close()
sns.lineplot(data=training_df[training_df.bs=='2000'], x='Epoch', y='Negative log-likelihood', hue='Optimizer')
plt.show()
plt.savefig(f'figures/experiment_1_2000.png')

In [ ]:
plt.close()
sns.lineplot(data=training_df[training_df.bs=='4000'], x='Epoch', y='Negative log-likelihood', hue='Optimizer')
plt.show()
plt.savefig(f'figures/experiment_1_4000.png')

In [ ]:
plt.close()
sns.lineplot(data=training_df[training_df.bs=='6000'], x='Epoch', y='Negative log-likelihood', hue='Optimizer')
plt.show()
plt.savefig(f'figures/experiment_1_6000.png')